In [1]:
#Import ALl the libaries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
#Reading the csv file import the data set
train_df=pd.read_csv("train.csv", nrows=2000000)

In [3]:
train_df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [4]:
#Fuction To remove the Outliers fro train_dataset
def remove_outliers_for_train(df):
    return df[(df['fare_amount'] >= 1.) & 
              (df['fare_amount'] <= 500.) &
              (df['pickup_longitude'] >= -75) & 
              (df['pickup_longitude'] <= -72) & 
              (df['dropoff_longitude'] >= -75) & 
              (df['dropoff_longitude'] <= -72) & 
              (df['pickup_latitude'] >= 40) & 
              (df['pickup_latitude'] <= 42) & 
              (df['dropoff_latitude'] >=40) & 
              (df['dropoff_latitude'] <= 42) & 
              (df['passenger_count'] >= 1) & 
              (df['passenger_count'] <= 5)]

In [5]:
#before removeing outliers
train_df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,2.000000e+06,2.000000e+06,2.000000e+06,1.999986e+06,1.999986e+06,2.000000e+06
mean,1.134779e+01,-7.252321e+01,3.992963e+01,-7.252395e+01,3.992808e+01,1.684113e+00
std,9.852883e+00,1.286804e+01,7.983352e+00,1.277497e+01,1.032382e+01,1.314982e+00
min,-6.200000e+01,-3.377681e+03,-3.458665e+03,-3.383297e+03,-3.461541e+03,0.000000e+00
25%,6.000000e+00,-7.399208e+01,4.073491e+01,-7.399141e+01,4.073400e+01,1.000000e+00
50%,8.500000e+00,-7.398181e+01,4.075263e+01,-7.398016e+01,4.075312e+01,1.000000e+00
75%,1.250000e+01,-7.396713e+01,4.076710e+01,-7.396369e+01,4.076809e+01,2.000000e+00
max,1.273310e+03,2.856442e+03,2.621628e+03,3.414307e+03,3.345917e+03,2.080000e+02


In [6]:
#calling function
train_df=remove_outliers_for_train(train_df)

In [7]:
#After Removing the outliers
train_df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.910120e+06,1.910120e+06,1.910120e+06,1.910120e+06,1.910120e+06,1.910120e+06
mean,1.132295e+01,-7.397518e+01,4.075102e+01,-7.397429e+01,4.075137e+01,1.597499e+00
std,9.738299e+00,3.924795e-02,3.012578e-02,3.841298e-02,3.322951e-02,1.154000e+00
min,2.000000e+00,-7.496814e+01,4.005272e+01,-7.496426e+01,4.001931e+01,1.000000e+00
25%,6.000000e+00,-7.399229e+01,4.073652e+01,-7.399160e+01,4.073551e+01,1.000000e+00
50%,8.500000e+00,-7.398210e+01,4.075333e+01,-7.398062e+01,4.075383e+01,1.000000e+00
75%,1.250000e+01,-7.396835e+01,4.076751e+01,-7.396537e+01,4.076839e+01,2.000000e+00
max,5.000000e+02,-7.206320e+01,4.192279e+01,-7.219609e+01,4.195112e+01,5.000000e+00


In [8]:
#checking the info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1910120 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 131.2+ MB


In [9]:
#function to calculate distance in km
def haversine_np(lon1, lat1, lon2, lat2):
    
    
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [10]:
#Adding Distance column in train_df
train_df['distance(km)'] = haversine_np(train_df['pickup_longitude'], train_df['pickup_latitude'], train_df['dropoff_longitude'], train_df['dropoff_latitude'])

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1910120 entries, 0 to 1999999
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
 8   distance(km)       float64
dtypes: float64(6), int64(1), object(2)
memory usage: 145.7+ MB


In [12]:
train_df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance(km)
count,1.910120e+06,1.910120e+06,1.910120e+06,1.910120e+06,1.910120e+06,1.910120e+06,1.910120e+06
mean,1.132295e+01,-7.397518e+01,4.075102e+01,-7.397429e+01,4.075137e+01,1.597499e+00,3.329032e+00
std,9.738299e+00,3.924795e-02,3.012578e-02,3.841298e-02,3.322951e-02,1.154000e+00,3.801133e+00
min,2.000000e+00,-7.496814e+01,4.005272e+01,-7.496426e+01,4.001931e+01,1.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399229e+01,4.073652e+01,-7.399160e+01,4.073551e+01,1.000000e+00,1.252854e+00
50%,8.500000e+00,-7.398210e+01,4.075333e+01,-7.398062e+01,4.075383e+01,1.000000e+00,2.150865e+00
75%,1.250000e+01,-7.396835e+01,4.076751e+01,-7.396537e+01,4.076839e+01,2.000000e+00,3.912205e+00
max,5.000000e+02,-7.206320e+01,4.192279e+01,-7.219609e+01,4.195112e+01,5.000000e+00,1.615847e+02


In [13]:
#Reading the test_dataset
test_df=pd.read_csv("test.csv")

In [14]:
#before removing outliers
test_df.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


In [15]:
#function for test_df to remove outliers
def remove_outliers_for_test(df):
    return df[(df['pickup_longitude'] >= -75) & 
              (df['pickup_longitude'] <= -72) & 
              (df['dropoff_longitude'] >= -75) & 
              (df['dropoff_longitude'] <= -72) & 
              (df['pickup_latitude'] >= 40) & 
              (df['pickup_latitude'] <= 42) & 
              (df['dropoff_latitude'] >=40) & 
              (df['dropoff_latitude'] <= 42) & 
              (df['passenger_count'] >= 1) & 
              (df['passenger_count'] <= 5)]

In [16]:
test_df=remove_outliers_for_test(test_df)

In [17]:
#After removing outliers
test_df.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9737.000000,9737.000000,9737.000000,9737.000000,9737.000000
mean,-73.974696,40.751077,-73.973678,40.751719,1.592585
std,0.042870,0.033662,0.039094,0.035433,1.148075
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992480,40.736167,-73.991221,40.735257,1.000000
50%,-73.982277,40.753079,-73.980000,40.754052,1.000000
75%,-73.967935,40.767152,-73.964172,40.768732,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,5.000000


In [18]:
#Adding the Distance column in test_df
test_df['distance(km)'] = haversine_np(test_df['pickup_longitude'], test_df['pickup_latitude'], test_df['dropoff_longitude'], test_df['dropoff_latitude'])

In [19]:
test_df.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance(km)
count,9737.000000,9737.000000,9737.000000,9737.000000,9737.000000,9737.000000
mean,-73.974696,40.751077,-73.973678,40.751719,1.592585,3.427607
std,0.042870,0.033662,0.039094,0.035433,1.148075,3.972685
min,-74.252193,40.573143,-74.263242,40.568973,1.000000,0.000000
25%,-73.992480,40.736167,-73.991221,40.735257,1.000000,1.293287
50%,-73.982277,40.753079,-73.980000,40.754052,1.000000,2.215105
75%,-73.967935,40.767152,-73.964172,40.768732,2.000000,4.038986
max,-72.986532,41.709555,-72.990963,41.696683,5.000000,99.933258


In [20]:
#Traing The model

In [21]:
features = ['distance(km)','passenger_count']
label= 'fare_amount'

In [22]:
X=train_df[features]
y=train_df[label]

In [23]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=77)

In [24]:
lm=LinearRegression()

In [25]:
lm.fit(X_train,y_train)

LinearRegression()

In [26]:
prediction=lm.predict(X_val)

In [27]:
#Taking the Root mean Squared Error
rmse = np.sqrt(mean_squared_error(y_val, prediction))

In [31]:
print("RMSE:",rmse)

RMSE: 5.612261048563226


In [33]:
prediction_test=lm.predict(test_df[features])

In [34]:
test_df["fare_amount"]=prediction_test

In [ ]:
#Making Sub_df from submission

In [38]:
sub_df=test_df[["key","fare_amount"]]

In [39]:
sub_df.head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,9.237303
1,2015-01-27 13:08:24.0000003,9.448442
2,2011-10-08 11:53:44.0000002,5.711960
3,2012-12-01 21:12:12.0000002,8.488183
4,2012-12-01 21:12:12.0000003,15.574037


In [40]:
#Saving the sub_df into files


In [41]:
sub_df.to_csv("Submission.csv",index=None)